# M3.4 - Analyzing a Global Precipitation Data Cube

*Part of:* [**Open Science for Water Resources**](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources)

In [ ]:
import datetime
import earthaccess
import numpy as np
import h5py
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

$$
P = E + R + \Delta S
$$

![](./assets/water_budget.png)

[*Image courtesy of the USGS*](https://www.usgs.gov/media/images/components-a-simple-water-budget-part-a-watershed)

--- 

## Estimating evapotranspiration for our basin

In [ ]:
import geopandas

basin = geopandas.read_file('/home/arthur.endsley/Workspace/NTSG/projects/Y2024_TOPS_Training/data/YellowstoneRiver_drainage_WSG84.shp')

In [ ]:
basin.bounds.to_numpy().ravel()

In [ ]:
tuple(basin.bounds.to_numpy().ravel())

In [ ]:
# https://dx.doi.org/10.5067/MODIS/MOD16A3GF.061

results = earthaccess.search_data(
    short_name = 'MOD16A3GF',
    temporal = ('2014-01-01', '2018-12-31'),
    bounding_box = tuple(basin.bounds.to_numpy().ravel()))

In [ ]:
results_clean = list(filter(lambda granule: 'h10v04' in granule['meta']['native-id'], results))

In [ ]:
earthaccess.download(results_clean, 'data/MOD16A3')

## Installing a new Python package

In [ ]:
!pip install py4eos

In [ ]:
import py4eos

hdf = py4eos.read_hdf4eos('data/MOD16A3/MOD16A3GF.A2014001.h10v04.061.2022077145846.hdf')
hdf

In [ ]:
et = hdf.get('ET_500m')
et

In [ ]:
et = hdf.get('ET_500m', scale_and_offset = True)
et

---

## Obtaining basin-scale runoff data

In [ ]:
import xarray as xr

ds = xr.open_dataset('/home/arthur/Workspace/NTSG/projects/Y2024_TOPS_Training/data/HYSETS_watershed_YellowstoneRiver.nc')
ds

In [ ]:
ds_4years = ds.sel(time = slice('2014-01-01', '2018-12-31'))
ds_4years

In [ ]:
ds_4years['discharge'].plot()

In [ ]:
# Compute total daily discharge based on the mean rate
ds_4years['discharge_total'] = ds_4years.discharge * 60 * 60 * 24

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

ds_4years['discharge_total'].resample(time = 'YS').sum().to_series().plot.bar()